# Biblioteca Digital

In [98]:
from datetime import datetime, timedelta

class Material:
    def __init__(self, estado="disponible"):
        self.estado = estado 

In [99]:
class Libro(Material):
    def __init__(self, autor, genero, estado="disponible"):
        super().__init__(estado)
        self.autor = autor
        self.genero = genero
        

In [100]:
class Revista(Material):
    def __init__(self, edicion, periodicidad, estado="disponible"):
        super().__init__(estado)
        self.edicion = edicion
        self.periodicidad = periodicidad

In [101]:
class MaterialDigital(Material):
    def __init__(self, tipo_archivo, enlace_descarga, estado="disponible"):
        super().__init__(estado)
        self.tipo_archivo = tipo_archivo
        self.enlace_descarga = enlace_descarga

In [102]:
class Persona:
        def __init__(self, nombre):
            self.nombre = nombre

In [103]:
class Usuario(Persona):
    def __init__(self, nombre):
        super().__init__(nombre)
        self.materiales_prestados = []

    def consultar_catalogo(self, catalogo, autor=None, genero=None, tipo=None):
        return catalogo.buscar_materiales(autor=autor, genero=genero, tipo=tipo)

    def prestar_material(self, material, fecha_devolucion):
        if material.estado == "disponible":
            self.materiales_prestados.append((material, fecha_devolucion))
            material.estado = "prestado"
            return True
        return False

    def devolver_material(self, material):
        for m, _ in self.materiales_prestados:
            if m == material:
                material.estado = "disponible"
                self.materiales_prestados.remove((m, _))
                return True
        return False

In [104]:
class Bibliotecario(Persona):
    def __init__(self, nombre):
        super().__init__(nombre)

    def agregar_material(self, material, sucursal):
        sucursal.agregar_material(material)

    def gestionar_prestamos(self, prestamo, sucursal):
        sucursal.registrar_prestamo(prestamo)


In [105]:
class Catalogo:
    def __init__(self):
        self.materiales = []

    def agregar_material(self, material):
        self.materiales.append(material)

    def eliminar_material(self, material):
        if material in self.materiales:
            self.materiales.remove(material)

    def buscar_materiales(self, autor=None, genero=None, tipo=None):
        resultados = []
        for material in self.materiales:
            if isinstance(material, Libro) and autor and material.autor == autor:
                resultados.append(material)
            elif isinstance(material, Revista) and genero and material.genero == genero:
                resultados.append(material)
            elif isinstance(material, MaterialDigital) and tipo and material.tipo_archivo == tipo:
                resultados.append(material)
        return resultados

In [106]:

class Sucursal:
    def __init__(self, nombre):
        self.nombre = nombre
        self.catalogo = Catalogo()
        self.prestamos = []

    def agregar_material(self, material):
        self.catalogo.agregar_material(material)

    def registrar_prestamo(self, prestamo):
        self.prestamos.append(prestamo)

    def transferir_material(self, material, sucursal_destino):
        if material in self.catalogo.materiales:
            self.catalogo.eliminar_material(material)
            sucursal_destino.agregar_material(material)
            print(f"El material '{material.__class__.__name__}' ha sido transferido de {self.nombre} a {sucursal_destino.nombre}.")
        else:
            print(f"El material '{material.__class__.__name__}' no está en el catálogo de {self.nombre}.")

    def consultar_catalogo(self):
        return self.catalogo.materiales

In [107]:
class Prestamo:
    def __init__(self, usuario, material, fecha_prestamo, fecha_devolucion):
        self.usuario = usuario
        self.material = material
        self.fecha_prestamo = fecha_prestamo
        self.fecha_devolucion = fecha_devolucion

    def calcular_penalizacion(self):
        if datetime.now() > self.fecha_devolucion:
            return (datetime.now() - self.fecha_devolucion).days * 5
        return 0


In [108]:
class Penalizacion:
    def __init__(self, usuario):
        self.usuario = usuario
        self.multa = 0

    def agregar_penalizacion(self, monto):
        self.multa += monto

    def mostrar_penalizacion(self):
        if self.multa > 0:
            print(f"El usuario {self.usuario.nombre} tiene una penalizacion de ${self.multa}")
        else:
            print(f"El usuario {self.usuario.nombre} no tiene penalización")

In [109]:
libro1 = Libro(autor="Thomas Harris", genero="Thriller psicológico")
libro2 = Libro(autor="Dante Aligheri", genero="Ficcion Universal")
revista1 = Revista(edicion=8, periodicidad="Mensual")
material_digital1 = MaterialDigital(tipo_archivo="PDF", enlace_descarga="www.Hogarlibro.com")

sucursal1 = Sucursal(nombre="Surcusal Central")
sucursal2 = Sucursal(nombre="Sucursal Norte")
# Crear el catálogo
sucursal1.agregar_material(libro1)
sucursal1.agregar_material(libro2)
sucursal1.agregar_material(revista1)
sucursal1.agregar_material(material_digital1)

# Crear un usuario
usuario1 = Usuario(nombre="Carlos")

# Mostrar todos los materiales disponibles en el catálogo
print("Consulta general del catálogo:")
for material in sucursal1.consultar_catalogo():
    if isinstance(material, Libro):
        print(f"Libro: Autor: {material.autor}, Género: {material.genero}")
    elif isinstance(material, Revista):
        print(f"Revista: Edición: {material.edicion}, Periodicidad: {material.periodicidad}")
    elif isinstance(material, MaterialDigital):
        print(f"Material Digital: Tipo de archivo: {material.tipo_archivo}, Enlace: {material.enlace_descarga}")
# Usuario presta material
fecha_devolucion = datetime.now() + timedelta(days=7)
usuario1.prestar_material(libro1, fecha_devolucion)

# Crear préstamo
prestamo1 = Prestamo(usuario1, libro1, datetime.now() - timedelta(days=10), fecha_devolucion=fecha_devolucion)

# Crear penalización
penalizacion = Penalizacion(usuario1)
penalizacion.agregar_penalizacion(prestamo1.calcular_penalizacion())
penalizacion.mostrar_penalizacion()

# Transferir material entre sucursales
sucursal1.transferir_material(libro1, sucursal2)

# Mostrar el catálogo después de la transferencia
print("\nCatálogo después de la transferencia:")
print("\nSucursal Central:")
for material in sucursal1.consultar_catalogo():
    if isinstance(material, Libro):
        print(f"Libro: Autor: {material.autor}, Género: {material.genero}")
    elif isinstance(material, Revista):
        print(f"Revista: Edición: {material.edicion}, Periodicidad: {material.periodicidad}")
    elif isinstance(material, MaterialDigital):
        print(f"Material Digital: Tipo de archivo: {material.tipo_archivo}, Enlace: {material.enlace_descarga}")
print("\nSucursal Norte:")
for material in sucursal2.consultar_catalogo():
    if isinstance(material, Libro):
        print(f"Libro: Autor: {material.autor}, Género: {material.genero}")
    elif isinstance(material, Revista):
        print(f"Revista: Edición: {material.edicion}, Periodicidad: {material.periodicidad}")
    elif isinstance(material, MaterialDigital):
        print(f"Material Digital: Tipo de archivo: {material.tipo_archivo}, Enlace: {material.enlace_descarga}")

Consulta general del catálogo:
Libro: Autor: Thomas Harris, Género: Thriller psicológico
Libro: Autor: Dante Aligheri, Género: Ficcion Universal
Revista: Edición: 8, Periodicidad: Mensual
Material Digital: Tipo de archivo: PDF, Enlace: www.Hogarlibro.com
El usuario Carlos no tiene penalización
El material 'Libro' ha sido transferido de Surcusal Central a Sucursal Norte.

Catálogo después de la transferencia:

Sucursal Central:
Libro: Autor: Dante Aligheri, Género: Ficcion Universal
Revista: Edición: 8, Periodicidad: Mensual
Material Digital: Tipo de archivo: PDF, Enlace: www.Hogarlibro.com

Sucursal Norte:
Libro: Autor: Thomas Harris, Género: Thriller psicológico
